In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import math
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from keras.models import Model
import keras.backend as K
import os
print("Running Tensorflow version", tf.__version__)

Running Tensorflow version 2.1.0


Using TensorFlow backend.


In [2]:
class Round(Layer):
    def __init__(self, **kwargs):
        super(Round, self).__init__(**kwargs)
    def get_output(self, train=False):
        X = self.get_input(train)
        return K.round(X * 4096) / 4096.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Round, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class Integer(Layer):
    def __init__(self, **kwargs):
        super(Integer, self).__init__(**kwargs)
    def get_output(self, train=False):
        X = self.get_input(train)
        return X / 4096.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Integer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
def DBL(previousLayer, layerFilter, kernelSize=(3, 3)):
    return LeakyReLU()(BatchNormalization()(Conv2D(filters=layerFilter, kernel_size=kernelSize, padding='same')(previousLayer)))
print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
classificationClass = 4
print("Number of class classification is", classificationClass)

Number of class classification is 4


In [4]:
model_0_input = Input(shape=(448, 448, 3))
# 448 x 448 x 3
model_0_startBranch = DBL(previousLayer=model_0_input, layerFilter=16) 
# 448 x 448 x 16
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# 224 x 224 x 16
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=32)
# 224 x 224 x 32
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# 112 x 112 x 32
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=64)
# 112 x 112 x 64
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# 56 x 56 x 64
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=128)
# 56 x 56 x 128
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# 28 x 28 x 128
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=256)
# 28 x 28 x 256
# 2 branch split from startBranch (28 x 28 x 256), following model_0_branch0
model_0_branch0 = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# 14 x 14 x 256
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=512)
# 14 x 14 x 512
model_0_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_0_branch0)
# 14 x 14 x 512
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=1024)
# 14 x 14 x 1024
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=256, kernelSize=(1, 1))
# 14 x 14 x 256
# 2 branch split from model_0_branch0 (14 x 14 x 256), following model_0_branch00
model_0_branch00 = DBL(previousLayer=model_0_branch0, layerFilter=128, kernelSize=(1, 1))
# 14 x 14 x 128
model_0_branch00 = UpSampling2D()(model_0_branch00)
# 28 x 28 x 128
# 2 branch merge from model_0_branch1 (unchanged from model_0_startBranch) and model_0_branch00
model_0_mergedBranch = Concatenate()([model_0_startBranch, model_0_branch00])
# 28 x 28 x 384
model_0_mergedBranch = DBL(previousLayer=model_0_mergedBranch, layerFilter=256)
# 28 x 28 x 256
model_0_mergedBranch = DBL(previousLayer=model_0_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
# 28 x 28 x (3 * (5 + classificationClass))
# OUTPUT = model_0_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

# following model_0_branch01
model_0_branch01 = DBL(previousLayer=model_0_branch0, layerFilter=512)
# 14 x 14 x 512
model_0_branch01 = DBL(previousLayer=model_0_branch01, layerFilter=3 * (4 + 1 + classificationClass))
# 14 x 14 x (3 * (5 + classificationClass))
# OUTPUT = model_0_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_0 = Model(inputs=model_0_input, outputs=[model_0_mergedBranch, model_0_branch01])
model_0.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights=[1., 0.2])
print("Model model_0 compilation complete")

Model model_0 compilation complete
